<a href="https://colab.research.google.com/github/ibahas/Uploadee-Google-Drive/blob/main/Colab_Multi_Destination_File_Uploader_(Upload_ee_%26_Google_Drive).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab Multi-Destination File Uploader (Upload.ee & Google Drive)

This notebook provides an interactive file browser and uploader GUI within Google Colaboratory.

**Features:**
*   Browse `/content/`.
*   Select multiple files.
*   Upload selected files sequentially to **Upload.ee** or **Google Drive**.
*   Progress bar for current file upload.
*   Status logging.

**Instructions:**
1.  Run the **"Install Dependencies"** cell below first.
2.  Run the **"Main Application Code"** cell to launch the GUI.
3.  Follow the usage instructions within the application's output area or the README.md if available.

# Install necessary libraries (run this cell first)
!pip install --upgrade requests google-api-python-client google-auth-httplib2 google-auth-oauthlib --quiet
print("Dependencies installed/updated.")

---


## Main Application Code

Run the following cell to start the file browser and uploader interface. The UI widgets will appear below this cell after execution.

In [6]:
# ======================================================================
# Colab File Browser & Multi-Destination Uploader (Multiple Files)
# v4.12 - Display Upload.ee link on success
# ======================================================================

# --- Imports ---
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import html
import subprocess
import shlex
import re
import requests
import time
import json
from functools import partial # For cleaner event handlers

# --- Google Drive Specific Imports ---
try:
    from google.colab import auth as colab_auth
    from google.auth import default as google_auth_default
    from googleapiclient.discovery import build as build_google_service
    from googleapiclient.errors import HttpError
    from googleapiclient.http import MediaFileUpload
    google_libs_available = True
except ImportError:
    google_libs_available = False
    print("⚠️ Warning: Google Drive libraries not found. Run 'pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib'")

# --- Global State ---
current_path = '/content/'
selected_files_list = [] # Use a list for multiple selections
# --- Google Drive State ---
drive_service = None
drive_authenticated = False
# --- UI References ---
file_checkboxes = {} # Dictionary to store checkboxes {path: checkbox_widget}

# ======================================================
#  Helper Function: Get Upload.ee Details
# ======================================================
# (Unchanged from previous working version)
def get_upload_ee_details():
    """
    Connects to upload.ee to fetch dynamic upload ID, URL, and cookies.
    Returns: tuple: (dynamic_upload_url, cookie_string, upload_id) or (None, None, None) on failure.
    """
    # print("Attempting to fetch Upload.ee dynamic upload ID...") # Less verbose
    session = requests.Session()
    link_script_url = "https://www.upload.ee/ubr_link_upload.php"
    base_url = "https://www.upload.ee/"
    rnd_id = int(time.time() * 1000); params = {'rnd_id': rnd_id}
    headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36', 'Accept-Language': 'en-US,en;q=0.9', }
    session.headers.update(headers)
    try: # Pre-load main page for cookies
        preload_headers = { 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Sec-Fetch-Dest': 'document','Sec-Fetch-Mode': 'navigate', 'Sec-Fetch-Site': 'same-origin', 'Upgrade-Insecure-Requests': '1', 'Referer': base_url, }
        session.headers.update(preload_headers); main_page_response = session.get(base_url, timeout=15); main_page_response.raise_for_status()
    except Exception as e: print(f"⚠️ Warn: Failed pre-fetch: {e}") # Continue even if pre-fetch fails
    script_headers = { 'Accept': '*/*', 'Referer': base_url, 'Sec-Fetch-Dest': 'script', 'Sec-Fetch-Mode': 'no-cors', 'Sec-Fetch-Site': 'same-origin', }
    session.headers.update(script_headers); upload_id = None
    try: # Fetch Upload ID
        response = session.get(link_script_url, params=params, timeout=15); response.raise_for_status()
        match = re.search(r'startUpload\("([a-f0-9]+)"\s*,\s*0\s*\);', response.text)
        if match: upload_id = match.group(1);
        else: print("❌ Failed to find upload_id pattern."); return None, None, None
        dynamic_upload_url = f"https://www.upload.ee/cgi-bin/ubr_upload.pl?X-Progress-ID={upload_id}&upload_id={upload_id}"
        cookie_dict = session.cookies.get_dict(); cookie_string = "; ".join([f"{k}={v}" for k, v in cookie_dict.items()])
        return dynamic_upload_url, cookie_string, upload_id
    except Exception as e: print(f"❌ Error fetching Upload.ee ID: {type(e).__name__} - {e}"); return None, None, None

# ======================================================
#  Helper Function: Authenticate Google Drive
# ======================================================
# (Unchanged from previous working version)
def authenticate_google_drive():
    """Handles Google Drive authentication using Colab."""
    global drive_service, drive_authenticated, output_area
    if not google_libs_available:
        with output_area: print("❌ Google libraries not installed."); return False
    try:
        with output_area: clear_output(wait=True); print("Authenticating Google Drive via Colab...")
        colab_auth.authenticate_user()
        creds, _ = google_auth_default()
        drive_service = build_google_service('drive', 'v3', credentials=creds)
        drive_authenticated = True
        with output_area: clear_output(wait=True); print("✅ Google Drive Authentication Successful!")
        return True
    except Exception as e:
        drive_authenticated = False; drive_service = None
        with output_area: clear_output(wait=True); print(f"❌ Google Drive Authentication Failed: {type(e).__name__} - {e}")
        return False

# ======================================================
#  Helper Function: Upload to Google Drive
# ======================================================
# (Unchanged from previous working version)
def perform_google_drive_upload(file_path, file_name, target_folder_id):
    """Uploads the specified file to Google Drive."""
    global drive_service, drive_authenticated, output_area, progress_bar
    if not drive_authenticated or drive_service is None: print("⚠️ Please Authenticate Google Drive first!"); return False
    print(f"\nStarting Google Drive upload for: {file_name}")
    progress_bar.value = 0; progress_bar.description = f'GDrive: {file_name[:20]}...'; progress_bar.bar_style = 'info'; progress_bar.layout.display = 'block'
    try:
        file_metadata = {'name': file_name}
        if target_folder_id:
             if re.match(r'^[a-zA-Z0-9_-]{10,}$', target_folder_id): file_metadata['parents'] = [target_folder_id]; print(f"   Targeting Folder ID: {target_folder_id}")
             else: print(f"⚠️ Invalid Folder ID format: '{target_folder_id}'. Uploading to root.")
        else: print("   No Folder ID specified. Uploading to root 'My Drive'.")
        media = MediaFileUpload(file_path, chunksize=1024*1024*5, resumable=True)
        request = drive_service.files().create(body=file_metadata, media_body=media, fields='id, name, webViewLink')
        response = None; print("   Uploading..."); last_progress_percent = 0
        while response is None:
            status, response = request.next_chunk()
            if status: percent = int(status.progress() * 100); progress_bar.value = percent
        progress_bar.value = 100 # Ensure 100%
        file_id = response.get('id'); file_name_uploaded = response.get('name'); file_link = response.get('webViewLink')
        print(f"✅ GDrive Upload Successful: {file_name_uploaded} ({file_id})"); print(f"   View Link: {file_link}")
        progress_bar.bar_style = 'success'; return True
    except HttpError as error:
        print(f"❌ Google Drive API Error for {file_name}: {error}")
        try: error_details = json.loads(error.content).get('error'); print(f"   Reason: {error_details.get('message')} (Code: {error_details.get('code')})")
        except Exception: pass
        progress_bar.bar_style = 'danger'; return False
    except Exception as e: print(f"❌ Unexpected Error during GDrive upload for {file_name}: {type(e).__name__} - {e}"); progress_bar.bar_style = 'danger'; return False

# ======================================================
#  Helper Function: Upload to Upload.ee (Returns Link)
# ======================================================
def perform_upload_ee_upload(file_path, file_name, upload_url, cookie_header_value, upload_id):
    """
    Handles the Upload.ee upload process with progress polling.
    Returns the 'finished page' URL string on success, None on failure.
    """
    global output_area, progress_bar

    print(f"\nStarting Upload.ee upload for: {file_name}")
    progress_bar.value = 0; progress_bar.description = f'Upload.ee: {file_name[:18]}...'; progress_bar.bar_style = 'info'; progress_bar.layout.display = 'block'
    upload_process = None; success_status = False; final_url = None

    try:
        command = [ 'curl', '-L', upload_url, '-H', 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', '-H', 'Accept-Language: en-US,en;q=0.9', '-H', 'Cache-Control: max-age=0', '-H', 'Origin: https://www.upload.ee', '-H', 'Referer: https://www.upload.ee/', '-H', 'Sec-Fetch-Dest: iframe', '-H', 'Sec-Fetch-Mode: navigate', '-H', 'Sec-Fetch-Site: same-origin', '-H', 'Sec-Fetch-User: ?1', '-H', 'Upgrade-Insecure-Requests: 1', '-H', 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36', '-F', f'upfile_0=@{file_path};filename={file_name}', '-F', 'link=', '-F', 'email=', '-F', 'category=cat_file', '-F', 'big_resize=none', '-F', 'small_resize=120x90' ]
        if cookie_header_value: command.extend(['-b', cookie_header_value])
        user_agent_string = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'
        print("--- Starting Background Upload ---"); upload_process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, encoding='utf-8', errors='ignore')
        progress_url = "https://www.upload.ee/progress"
        progress_headers = {'User-Agent': user_agent_string,'Accept': '*/*', 'Referer': 'https://www.upload.ee/?', 'X-Progress-ID': upload_id,'Sec-Fetch-Dest': 'empty','Sec-Fetch-Mode': 'cors','Sec-Fetch-Site': 'same-origin',}
        progress_session = requests.Session(); progress_session.headers.update(progress_headers)
        if cookie_header_value:
            try: cookies_for_requests = {c.split('=')[0].strip(): c.split('=',1)[1].strip() for c in cookie_header_value.split(';') if '=' in c}; progress_session.cookies.update(cookies_for_requests)
            except Exception: pass
        last_progress = 0
        while upload_process.poll() is None:
            try:
                prog_response = progress_session.get(progress_url, timeout=5)
                if prog_response.status_code == 200:
                    try: data = prog_response.json(); state = data.get("state"); received = data.get("received"); size = data.get("size")
                    except (json.JSONDecodeError, AttributeError, TypeError): continue
                    if state == "uploading" and isinstance(received, int) and isinstance(size, int) and size > 0: percent = min(100.0, (received / size) * 100.0); progress_bar.value = percent
                    elif state == "done": break
                    elif state == "error": break
            except requests.exceptions.RequestException: pass
            time.sleep(1.5)
        print("--- Background Upload Finished ---"); progress_bar.value = 100
        stdout, stderr = upload_process.communicate(); return_code = upload_process.returncode
        server_error_found = False
        if stdout:
             success_pattern = r"parent\.location\.href='(https://www\.upload\.ee/\?page=finished&upload_id=([a-f0-9]+))'"
             match = re.search(success_pattern, stdout)
             if match:
                 success_status = True
                 final_url = match.group(1) # <<< Store the captured URL
             elif "Internal Server Error" in stdout or "Software error" in stdout: server_error_found = True
             elif "Forbidden" in stdout or "Error 403" in stdout: server_error_found = True
             elif "Payload Too Large" in stdout or "Error 413" in stdout: server_error_found = True

        # --- Updated Verdict Logic ---
        if success_status:
            print(f"✅ Upload.ee Successful: {file_name}")
            progress_bar.bar_style = 'success'
            return final_url # <<< RETURN URL ON SUCCESS
        elif server_error_found:
            print(f"❌ Upload.ee Failed (Server Error): {file_name}")
            progress_bar.bar_style = 'danger'
        elif return_code != 0:
            print(f"❌ Upload.ee Failed (Curl Error {return_code}): {file_name}")
            progress_bar.bar_style = 'danger'
        else:
            print(f"⚠️ Upload.ee Status Uncertain: {file_name}")
            progress_bar.bar_style = 'warning'

        return None # <<< RETURN None ON FAILURE/UNCERTAIN

    except Exception as e:
        print(f"❌ Unexpected Error during Upload.ee for {file_name}: {type(e).__name__} - {e}")
        if upload_process and upload_process.poll() is None:
             try: upload_process.kill(); upload_process.communicate()
             except Exception as kill_e: print(f"   (Error terminating process: {kill_e})")
        progress_bar.bar_style = 'danger'
        return None # <<< RETURN None ON EXCEPTION

# ======================================================
#  File Browser UI & Logic
# ======================================================
# (Unchanged)
def update_directory_view(path_to_display):
    global current_path, selected_files_list, output_area, dir_list_box, current_path_label
    global file_checkboxes, select_all_button, deselect_all_button # Need controls
    try: path_to_display = os.path.abspath(path_to_display)
    except Exception as e:
        with output_area: clear_output(wait=True); print(f"❌ Error resolving path: {e}")
        path_to_display = current_path if os.path.exists(current_path) else '/content/'
    if not path_to_display.startswith('/content'):
        with output_area: clear_output(wait=True); print(f"⚠️ Navigation restricted.")
        path_to_display = '/content/'
    if not os.path.exists(path_to_display) or not os.path.isdir(path_to_display):
         parent = os.path.dirname(current_path); path_to_display = parent if os.path.exists(parent) and os.path.isdir(parent) else '/content/'
         with output_area: clear_output(wait=True); print(f"ℹ️ Path not found/not dir. Resetting view.")
    current_path = path_to_display; current_path_label.value = f"<b>Current:</b> {html.escape(current_path)}"
    items = []; file_checkboxes.clear()
    try: items = sorted(os.listdir(current_path), key=str.lower)
    except PermissionError:
        with output_area: clear_output(wait=True); print(f"❌ Permission denied: {current_path}")
        dir_list_box.children = []; select_all_button.disabled=True; deselect_all_button.disabled=True; return
    except Exception as e:
        with output_area: clear_output(wait=True); print(f"❌ Error listing dir: {e}")
        dir_list_box.children = []; select_all_button.disabled=True; deselect_all_button.disabled=True; return
    new_children = []; current_dir_has_files = False
    item_layout = widgets.Layout(min_width='150px', max_width='200px', margin='5px', padding='5px', border='1px solid #ccc', display='flex', flex_direction='column', align_items='center')
    folder_button_layout = widgets.Layout(min_width='150px', max_width='200px', margin='5px', padding='5px', border='1px solid #a0a0ff', background_color='#e8e8ff')
    checkbox_layout = widgets.Layout(width='auto', margin='0 0 3px 0')
    label_layout = widgets.Layout(width='100%', text_align='center')
    if current_path != '/' and current_path != '/content': # Add "Go Up" Button
        parent_path = os.path.dirname(current_path)
        if parent_path.startswith('/content') or parent_path == '/':
             up_button = widgets.Button(description="⬆️ ..", layout=folder_button_layout, button_style='info'); up_button.tooltip = f"Go to: {parent_path}"
             up_button.on_click(lambda b, path=parent_path: update_directory_view(path)); new_children.append(up_button)
    for item in items: # Add Folders (as Buttons)
        item_path = os.path.join(current_path, item)
        try:
            if os.path.isdir(item_path):
                 folder_button = widgets.Button(description=f"📁 {item}", layout=folder_button_layout, button_style='primary'); folder_button.tooltip = f"Open: {item_path}"
                 folder_button.on_click(lambda b, path=item_path: update_directory_view(path)); new_children.append(folder_button)
        except Exception as e: print(f"⚠️ Warn: Cannot process folder {item}: {e}")
    for item in items: # Add Files (as Checkboxes + Labels)
        item_path = os.path.join(current_path, item)
        try:
            if not os.path.isdir(item_path) and os.path.exists(item_path):
                current_dir_has_files = True; display_name = item if len(item) < 20 else item[:17] + '...'
                file_label = widgets.Label(f"{display_name}", layout=label_layout, tooltip=item)
                checkbox = widgets.Checkbox(value=(item_path in selected_files_list), description='', indent=False, layout=checkbox_layout)
                file_checkboxes[item_path] = checkbox
                checkbox.observe(partial(handle_file_checkbox_change, item_path), names='value')
                file_entry = widgets.VBox([ widgets.HTML("📄", layout=widgets.Layout(margin='2px')), checkbox, file_label ], layout=item_layout)
                new_children.append(file_entry)
        except Exception as e: print(f"⚠️ Warn: Cannot process file {item}: {e}")
    dir_list_box.children = tuple(new_children)
    dir_list_box.layout.display = 'flex'; dir_list_box.layout.flex_flow = 'row wrap'; dir_list_box.layout.align_content = 'flex-start'; dir_list_box.layout.align_items = 'stretch'
    select_all_button.disabled = not current_dir_has_files; deselect_all_button.disabled = not current_dir_has_files

# --- UI Helper functions ---
# (Unchanged)
def update_selected_files_viewer():
    global selected_files_list, selected_files_viewer
    sorted_list = sorted([os.path.basename(p) for p in selected_files_list])
    selected_files_viewer.options = sorted_list; selected_files_viewer.value = tuple(sorted_list)
def handle_file_checkbox_change(path, change):
    global selected_files_list
    if change['new'] == True:
        if path not in selected_files_list: selected_files_list.append(path)
    else:
        if path in selected_files_list: selected_files_list.remove(path)
    update_selected_files_viewer()
def handle_select_all(b):
    global file_checkboxes
    for cb in file_checkboxes.values():
        if not cb.value: cb.value = True
def handle_deselect_all(b):
    global file_checkboxes
    for cb in file_checkboxes.values():
        if cb.value: cb.value = False

# ======================================================
#  Main Upload Button Handler (Router) - Updated for Link Display
# ======================================================
def handle_confirm_and_upload_click(b):
    global selected_files_list, output_area, progress_bar
    global upload_destination_dropdown, google_auth_button, gdrive_folder_id_input

    with output_area:
        try:
            clear_output(wait=True)
            if not selected_files_list: print("⚠️ No files selected for upload."); return

            destination = upload_destination_dropdown.value
            files_to_upload = list(selected_files_list) # Create copy
            total_files = len(files_to_upload)
            print(f"▶️ Preparing to upload {total_files} file(s) to {destination}...\n" + "-" * 20)

            target_folder_id_str = None
            if destination == 'Google Drive':
                if not google_libs_available: print("❌ Google libraries unavailable."); return
                if not drive_authenticated: print("✋ Please authenticate Google Drive first."); return
                target_folder_id_str = gdrive_folder_id_input.value.strip()

            confirm_and_upload_button.disabled = True; google_auth_button.disabled = True
            upload_success_count = 0; upload_fail_count = 0

            # Main Upload Loop
            for i, file_path in enumerate(files_to_upload):
                file_name = os.path.basename(file_path)
                print(f"\n--- Uploading file {i+1} of {total_files}: {file_name} ---")
                progress_bar.value = 0; progress_bar.layout.display = 'block'; upload_successful = False
                result_info = None # To store link or boolean

                try:
                    if destination == 'Upload.ee':
                        dynamic_upload_url, dynamic_cookie_string, upload_id = get_upload_ee_details()
                        if dynamic_upload_url and upload_id:
                            # <<< Store result (URL or None) >>>
                            result_info = perform_upload_ee_upload(file_path, file_name, dynamic_upload_url, dynamic_cookie_string, upload_id)
                            # <<< Check if result is a URL string for success >>>
                            upload_successful = isinstance(result_info, str)
                            if upload_successful:
                                print(f"   Finished Page URL: {result_info}") # Print the link
                        else:
                            print(f"❌ ERROR: Failed to get Upload.ee details for {file_name}.")
                            upload_successful = False # Explicitly set failure

                    elif destination == 'Google Drive':
                        # GDrive function returns True/False
                        upload_successful = perform_google_drive_upload(file_path, file_name, target_folder_id_str)
                        # result_info remains None for GDrive unless you want to store something else

                    # Update counts
                    if upload_successful: upload_success_count += 1
                    else: upload_fail_count += 1

                except Exception as e:
                    print(f"❌ UNEXPECTED ERROR during upload loop for {file_name}: {type(e).__name__}")
                    upload_fail_count += 1
                    progress_bar.bar_style = 'danger'

                # time.sleep(0.5) # Optional pause

            # Loop Finished - Final Report
            print("\n" + "=" * 30 + "\n      Upload Summary\n" + "=" * 30)
            print(f"Total Files Attempted: {total_files}\nSuccessful Uploads:    {upload_success_count}\nFailed Uploads:        {upload_fail_count}\n" + "=" * 30)

        finally:
             print("\n--- All uploads finished or failed (Cleanup). ---")
             time.sleep(1)
             progress_bar.layout.display = 'none'; progress_bar.value = 0; progress_bar.bar_style = 'info'
             confirm_and_upload_button.disabled = False
             google_auth_button.disabled = (upload_destination_dropdown.value != 'Google Drive')


# ======================================================
#  Google Drive Auth Button Handler
# ======================================================
# (Unchanged)
def handle_google_auth_click(b):
    global google_auth_button, upload_destination_dropdown
    google_auth_button.disabled = True; authenticate_google_drive()
    google_auth_button.disabled = (upload_destination_dropdown.value != 'Google Drive')

# ======================================================
#  Widget Creation
# ======================================================
# (Unchanged)
output_area = widgets.Output()
current_path_label = widgets.HTML(value=f"<b>Current:</b> {html.escape(current_path)}")
dir_list_box = widgets.VBox([], layout=widgets.Layout(width='100%', max_height='350px', overflow_y='auto', border='1px solid #ccc', padding='5px')) # Adjusted height
select_all_button = widgets.Button(description="Select All", button_style='info', layout=widgets.Layout(width='auto', margin='0 5px 0 0'), tooltip="Select all files currently listed", disabled=True)
deselect_all_button = widgets.Button(description="Deselect All", button_style='info', layout=widgets.Layout(width='auto'), tooltip="Deselect all files currently listed", disabled=True)
select_all_button.on_click(handle_select_all); deselect_all_button.on_click(handle_deselect_all)
select_buttons_box = widgets.HBox([select_all_button, deselect_all_button])
selected_files_viewer = widgets.SelectMultiple(options=[],value=[],description='Selected:',rows=5,disabled=False,layout=widgets.Layout(width='95%', min_height='80px'))
progress_bar = widgets.FloatProgress(value=0, min=0, max=100.0, description='Progress:', bar_style='info', orientation='horizontal', layout=widgets.Layout(width='95%', height='25px', display='none'))
upload_destination_dropdown = widgets.Dropdown(options=['Upload.ee', 'Google Drive'], value='Upload.ee', description='Destination:', disabled=False, layout=widgets.Layout(width='auto'))
gdrive_folder_id_input = widgets.Text(value='', placeholder='Optional: GDrive Folder ID', description='GDrive Folder:', disabled=True, layout=widgets.Layout(width='auto', flex='1'))
google_auth_button = widgets.Button(description="Authenticate GDrive", button_style='info', icon='google', disabled=True, layout=widgets.Layout(width='auto', margin='0 0 0 10px'), tooltip='Authorize access to Google Drive')
google_auth_button.on_click(handle_google_auth_click)
confirm_and_upload_button = widgets.Button(description="Upload Selected Files", button_style='success', icon='upload', layout=widgets.Layout(width='auto'), tooltip='Upload all selected files')
confirm_and_upload_button.on_click(handle_confirm_and_upload_click)

def on_destination_change(change): # Handles enabling/disabling GDrive widgets
    is_gdrive = (change['new'] == 'Google Drive'); gdrive_folder_id_input.disabled = not is_gdrive; google_auth_button.disabled = not is_gdrive
    if is_gdrive and not drive_authenticated: google_auth_button.disabled = False
    elif not is_gdrive: google_auth_button.disabled = True
upload_destination_dropdown.observe(on_destination_change, names='value')

# ======================================================
#  Layout
# ======================================================
# (Unchanged)
path_hbox = widgets.HBox([current_path_label], layout=widgets.Layout(align_items='center', margin='0 0 5px 0'))
destination_box = widgets.HBox([upload_destination_dropdown, gdrive_folder_id_input, google_auth_button], layout=widgets.Layout(align_items='center', flex_flow='row wrap', margin='5px 0'))
file_list_area = widgets.VBox([dir_list_box, select_buttons_box])
upload_control_area = widgets.VBox([selected_files_viewer, destination_box, progress_bar, confirm_and_upload_button], layout=widgets.Layout(margin='10px 0 0 0'))
gui_layout = widgets.VBox([ widgets.HTML("<b>File Browser & Uploader (Multi-Select)</b>"), path_hbox, file_list_area, widgets.HTML("<hr>"), upload_control_area, widgets.HTML("<hr><b>Messages & Upload Log:</b>"), output_area ], layout=widgets.Layout(border='1px solid black', padding='10px', width='95%', max_width='900px'))

# ======================================================
#  Initial Display
# ======================================================
# (Unchanged)
print("Initializing file browser...")
if not google_libs_available:
    print("\n *** Google Drive functionality disabled (libraries not found) *** \n")
    upload_destination_dropdown.options = ['Upload.ee']; upload_destination_dropdown.value = 'Upload.ee'
    gdrive_folder_id_input.disabled = True; google_auth_button.disabled = True
update_directory_view(current_path); update_selected_files_viewer(); display(gui_layout)
print("File browser ready.")
# --- End of Script ---

Initializing file browser...


File browser ready.
